# Multiword expression identification and extraction
Mateusz Wojtulewicz

In [1]:
import spacy
import tqdm

from pathlib import Path

## 1. Use SpaCy tokenizer API to tokenize the text from the law corpus.

In [2]:
nlp = spacy.load("pl_core_news_sm")
tokenizer = nlp.tokenizer

In [3]:
tokenized_acts = {}

acts_dir = Path("../data/ustawy/")
n_acts = len(list(acts_dir.iterdir()))

for act in tqdm.tqdm(acts_dir.iterdir(), desc="Tokenizing acts", total=n_acts):
    act_id = act.stem
    content = act.read_text(encoding="utf8")
    tokens = tokenizer(content)
    tokenized_acts[act_id] = [token.text.lower() for token in tokens]

Tokenizing acts: 100%|██████████| 1179/1179 [00:37<00:00, 31.82it/s]


## 2. Compute bigram counts of downcased tokens.

In [2]:
from collections import Counter

In [5]:
bigrams = (
    (f, s)
    for tokens in tokenized_acts.values()
    for (f, s) in zip(tokens[:-1], tokens[1:])
)

In [6]:
bigrams_count = Counter(bigrams)

In [7]:
bigrams_count.most_common(10)

[(('art', '.'), 83778),
 (('ust', '.'), 53552),
 (('.', '\n'), 49741),
 (('poz', '.'), 45198),
 ((',', 'poz'), 39655),
 (('-', '-'), 36542),
 (('r', '.'), 33015),
 (('w', 'art'), 30170),
 (('.', '1'), 29734),
 ((',', 'o'), 28739)]

## 3. Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts.

I'm using a Token's property `is_alpha` which is `True` if the token consists only of alphabetic characters.

In [3]:
def is_word(s: str) -> bool:
   return tokenizer(s)[0].is_alpha

In [9]:
bigrams_filtered = (
    (f, s)
    for tokens in tokenized_acts.values()
    for (f, s) in zip(tokens[:-1], tokens[1:])
    if is_word(f) and is_word(s)
)

In [10]:
bigrams_filtered_count = Counter(bigrams_filtered)

In [11]:
bigrams_filtered_count.most_common(20)

[(('w', 'art'), 30170),
 (('mowa', 'w'), 27649),
 (('w', 'ust'), 22238),
 (('których', 'mowa'), 12973),
 (('o', 'których'), 12604),
 (('otrzymuje', 'brzmienie'), 9168),
 (('z', 'dnia'), 8989),
 (('którym', 'mowa'), 8689),
 (('o', 'którym'), 8525),
 (('do', 'spraw'), 8215),
 (('dodaje', 'się'), 7960),
 (('i', 'nr'), 7871),
 (('w', 'brzmieniu'), 6690),
 (('w', 'drodze'), 6623),
 (('stosuje', 'się'), 6232),
 (('na', 'podstawie'), 5906),
 (('w', 'przypadku'), 5371),
 (('której', 'mowa'), 5192),
 (('o', 'której'), 5061),
 (('od', 'dnia'), 4971)]

## 4. Use pointwise mutual information to compute the measure for all pairs of words.
The _PMI_ of a pair of outcomes $x$ and $y$ is calculated as follows:
$$
\text{pmi}(x; y) = \log_2 \frac{p(x, y)}{p(x)p(y)}
$$

In our case:
- $p(x, y)$ is a probability of drawing a bigram $(x,y)$ from all bigrams in the corpus;
- $p(x)$ is a probabilty of drawing a token $x$ from all tokens in the corpus.

In [12]:
tokens_count = Counter(
    token for tokens in tokenized_acts.values() for token in tokens
)

In [13]:
tokens_count.most_common(10)

[('.', 437694),
 (',', 341126),
 ('w', 201224),
 ('\n', 181703),
 (')', 100194),
 ('i', 90009),
 ('art', 83804),
 ('z', 82443),
 ('1', 73108),
 ('o', 64776)]

Saving counters to be used to calculate _PMI_ in a `pmi_bigram_mp.py` script that uses parallelization.

In [9]:
import pickle

def dump_counter(counter: Counter, name: str) -> None:
    with open(f"saved/{name}.p", "wb") as f:
        pickle.dump(counter, f)

def load_counter(name) -> Counter:
    with open(f"saved/{name}.p", "rb") as f:
        return pickle.load(f)

In [17]:
dump_counter(bigrams_count, "bigrams_counter")
dump_counter(bigrams_filtered_count, "bigrams_filtered_counter")
dump_counter(tokens_count, "tokens_counter")

In [10]:
bigrams_count = load_counter("bigrams_counter")
bigrams_filtered_count = load_counter("bigrams_filtered_counter")
tokens_count = load_counter("tokens_counter")

Loading calculated results.

In [20]:
bigrams_pmi = pickle.load(open("saved/bigram_pmi_results.p", "rb"))

## 5. Sort the word pairs according to that measure in the descending order and determine top 10 entries.

In [21]:
sorted(bigrams_pmi, key=lambda x: -x[1])[:10]

[(('kołowe', 'jednoosiowe'), 22.476105450088575),
 (('zbrojeń', 'żelbeto'), 22.476105450088575),
 (('prefabrykatów', 'wnętrzowe'), 22.476105450088575),
 (('gołe', 'aluminiowe'), 22.476105450088575),
 (('polistyrenu', 'spienionego'), 22.476105450088575),
 (('objaśnieniem', 'figur'), 22.476105450088575),
 (('wkładzie', 'wnoszonym'), 22.476105450088575),
 (('doktorem', 'habilitowanym'), 22.476105450088575),
 (('losy', 'loteryjne'), 22.476105450088575),
 (('ugaszone', 'zapałki'), 22.476105450088575)]

## 6. Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5 occurrences).

In [22]:
bigrams_pmi_5 = [(bigram, pmi) for bigram, pmi in bigrams_pmi if bigrams_count[bigram] > 5]
sorted(bigrams_pmi_5, key=lambda x: -x[1])[:10]

[(('diagności', 'laboratoryjni'), 19.89114294936742),
 (('odczynów', 'poszczepiennych'), 19.89114294936742),
 (('adama', 'mickiewicza'), 19.89114294936742),
 (('papierem', 'wartościowym'), 19.668750528030973),
 (('piotrków', 'trybunalski'), 19.668750528030973),
 (('lambrekiny', 'okienne'), 19.668750528030973),
 (('schedę', 'spadkową'), 19.668750528030973),
 (('buraka', 'cukrowego'), 19.476105450088575),
 (('zdrowego', 'stylu'), 19.476105450088575),
 (('zapieczętowanej', 'kopercie'), 19.446358106694525)]

## 7. Use KRNNT or Clarin-PL API to tag and lemmatize the corpus.

I've used Clarin-PL API with Moreusz 1, and then I've downloaded the XML output files.

In [29]:
import xml.etree.ElementTree as ET

In [86]:
tag_tokenized_acts = {}

tag_acts_dir = Path("../data/ustawy_tagged_clarin_morf1/")
n_acts = len(list(tag_acts_dir.iterdir()))

for act in tqdm.tqdm(tag_acts_dir.iterdir(), desc="Tokenizing tagged acts", total=n_acts):
    act_id = act.stem
    content = act.read_text(encoding="utf8")
    tree = ET.fromstring(text=content)
    tag_tokens = []
    for tok in tree.iter("tok"):
        lex = tok.find("lex")
        base = lex.find("base").text
        ctag = lex.find("ctag").text
        tag_token = f"{base.lower()}:{ctag.split(':')[0]}"
        tag_tokens.append(tag_token)
    tag_tokenized_acts[act_id] = tag_tokens

Tokenizing tagged acts: 100%|██████████| 1179/1179 [02:16<00:00,  8.63it/s]


## 8. Using the tagged corpus compute bigram statistic for the tokens containing: a. lemmatized, downcased word b. morphosyntactic category of the word (subst, fin, adj, etc.)

In [87]:
tag_bigrams = [
    (f, s)
    for tag_tokens in tag_tokenized_acts.values()
    for (f, s) in zip(tag_tokens[:-1], tag_tokens[1:])
]

In [88]:
tag_bigrams_counter = Counter(tag_bigrams)

In [89]:
tag_bigrams_counter.most_common(10)

[(('art:brev', '.:interp'), 83700),
 (('usta:subst', '.:interp'), 53553),
 (('poz:brev', '.:interp'), 45195),
 ((',:interp', 'poz:brev'), 43166),
 (('.:interp', '1:adj'), 39987),
 (('-:interp', '-:interp'), 36548),
 (('r:brev', '.:interp'), 33029),
 (('w:prep', 'art:brev'), 31988),
 ((',:interp', 'o:prep'), 29906),
 (('o:prep', 'który:adj'), 28646)]

Now, I am filtering tokens that base is a word.

In [95]:
def is_word_base(token: str) -> bool:
    parts = token.split(":")
    if len(parts) != 2:
        return False
    else:
        return is_word(s=parts[0])

In [96]:
tag_bigrams_filtered = (
    (f, s)
    for tag_tokens in tag_tokenized_acts.values()
    for (f, s) in zip(tag_tokens[:-1], tag_tokens[1:])
    if is_word_base(f) and is_word_base(s)
)

In [97]:
tag_bigrams_filtered_counter = Counter(tag_bigrams_filtered)

In [99]:
tag_bigrams_filtered_counter.most_common(20)

[(('w:prep', 'art:brev'), 31988),
 (('o:prep', 'który:adj'), 28646),
 (('który:adj', 'mowa:subst'), 28538),
 (('mowa:subst', 'w:prep'), 28473),
 (('w:prep', 'usta:subst'), 23557),
 (('z:prep', 'dzień:subst'), 11360),
 (('otrzymywać:fin', 'brzmienie:subst'), 10536),
 (('określić:ppas', 'w:prep'), 10169),
 (('do:prep', 'sprawa:subst'), 8716),
 (('ustawa:subst', 'z:prep'), 8625),
 (('właściwy:adj', 'do:prep'), 8535),
 (('i:conj', 'nr:brev'), 8435),
 (('dodawać:fin', 'się:qub'), 8195),
 (('minister:subst', 'właściwy:adj'), 7931),
 (('w:prep', 'brzmienie:subst'), 7280),
 (('w:prep', 'droga:subst'), 7128),
 (('w:prep', 'przypadek:subst'), 6776),
 (('na:prep', 'podstawa:subst'), 6681),
 (('stosować:fin', 'się:qub'), 6535),
 (('się:qub', 'wyraz:subst'), 6077)]

## 10. Compute the same statistics as for the non-lemmatized words (i.e. PMI) and print top-10 entries with at least 5 occurrences.

In [100]:
tag_tokens_counter = Counter(
    tag_token for tag_tokens in tag_tokenized_acts.values() for tag_token in tag_tokens
)

In [101]:
tag_tokens_counter.most_common(10)

[('.:interp', 457356),
 (',:interp', 343058),
 ('w:prep', 202567),
 ('):interp', 102195),
 ('z:prep', 87991),
 ('i:conj', 87725),
 ('art:brev', 83706),
 ('1:adj', 74500),
 ('o:prep', 64712),
 ('-:interp', 61833)]

Saving counters to be used to calculate _PMI_ in a `pmi_bigram_mp.py` script that uses parallelization.

In [103]:
dump_counter(tag_bigrams_counter, "tag_bigrams_counter")
dump_counter(tag_bigrams_filtered_counter, "tag_bigrams_filtered_counter")
dump_counter(tag_tokens_counter, "tag_tokens_counter")

In [12]:
tag_bigrams_counter = load_counter("tag_bigrams_counter")
tag_bigrams_filtered_counter = load_counter("tag_bigrams_filtered_counter")
tag_tokens_counter = load_counter("tag_tokens_counter")

Loading calculated results.

In [17]:
tag_bigrams_pmi = pickle.load(open("saved/tag_bigram_pmi_results.p", "rb"))

Top 10 bigrams with any number of occurrences.

In [18]:
sorted(tag_bigrams_pmi, key=lambda x: -x[1])[:10]

[(('tornister:subst', 'nieskórzany:adj'), 22.33079552748708),
 (('zbrojenia:subst', 'żelbeto:adja'), 22.33079552748708),
 (('reduktor:subst', 'membranowy:adj'), 22.33079552748708),
 (('prefabrykat:subst', 'wnętrzowy:adj'), 22.33079552748708),
 (('polistyren:subst', 'spienić:ppas'), 22.33079552748708),
 (('uw:subst', 'zględnieniu:subst'), 22.33079552748708),
 (('english:subst', 'language:subst'), 22.33079552748708),
 (('language:subst', 'college:subst'), 22.33079552748708),
 (('southern:subst', 'trade:subst'), 22.33079552748708),
 (('łęka:subst', 'dukielski:adj'), 22.33079552748708)]

Top 10 bigrams with at least 5 occurrences.

In [19]:
tag_bigrams_pmi_5 = [(tag_bigram, pmi) for tag_bigram, pmi in tag_bigrams_pmi if tag_bigrams_counter[tag_bigram] > 5]
sorted(tag_bigrams_pmi_5, key=lambda x: -x[1])[:10]

[(('adam:subst', 'mickiewicz:subst'), 19.745833026765926),
 (('piotrków:subst', 'trybunalski:adj'), 19.523440605429478),
 (('przeponowy:adj', 'rurowy:adj'), 19.33079552748708),
 (('chrześcijanin:subst', 'baptysta:subst'), 19.108403106150632),
 (('maria:subst', 'curie:subst'), 19.00886743259972),
 (('hugo:subst', 'kołłątaj:subst'), 19.00886743259972),
 (('tadeusz:subst', 'kotarbiński:subst'), 19.00886743259972),
 (('upośledzony:adj', 'umysłowo:adv'), 18.871363908849784),
 (('jedwab:subst', 'wiskozowy:adj'), 18.871363908849784),
 (('vitis:subst', 'vinifera:subst'), 18.871363908849784)]

## 11. Compute **trigram** counts for both corpora and perform the same filtering.

### A. Non-lemmatized corpora.

In [107]:
trigrams_counter = Counter(
    (first, second, third)
    for tokens in tokenized_acts.values()
    for (first, second, third) in zip(tokens, tokens[1:], tokens[2:])
)

In [108]:
trigrams_counter.most_common(10)

[((',', 'poz', '.'), 39633),
 (('-', '-', '-'), 34641),
 (('w', 'art', '.'), 30162),
 (('ust', '.', '1'), 22618),
 (('w', 'ust', '.'), 22204),
 (('r', '.', 'nr'), 16956),
 (('_', '_', '_'), 16111),
 (('których', 'mowa', 'w'), 12506),
 (('mowa', 'w', 'ust'), 12388),
 ((',', 'o', 'których'), 12009)]

In [110]:
trigrams_filtered_counter = Counter(
    (first, second, third)
    for tokens in tokenized_acts.values()
    for (first, second, third) in zip(tokens, tokens[1:], tokens[2:])
    if is_word(first) and is_word(second) and is_word(third)
)

In [111]:
trigrams_filtered_counter.most_common(10)

[(('których', 'mowa', 'w'), 12506),
 (('mowa', 'w', 'ust'), 12388),
 (('o', 'których', 'mowa'), 11714),
 (('mowa', 'w', 'art'), 10995),
 (('którym', 'mowa', 'w'), 8429),
 (('o', 'którym', 'mowa'), 8040),
 (('której', 'mowa', 'w'), 5020),
 (('o', 'której', 'mowa'), 4731),
 (('właściwy', 'do', 'spraw'), 4434),
 (('minister', 'właściwy', 'do'), 4106)]

### B. Lemmatized and tagged corpora.

In [112]:
tag_trigrams_counter = Counter(
    (first, second, third)
    for tokens in tag_tokenized_acts.values()
    for (first, second, third) in zip(tokens, tokens[1:], tokens[2:])
)

In [113]:
tag_trigrams_counter.most_common(10)

[((',:interp', 'poz:brev', '.:interp'), 43165),
 (('-:interp', '-:interp', '-:interp'), 34646),
 (('w:prep', 'art:brev', '.:interp'), 31987),
 (('o:prep', 'który:adj', 'mowa:subst'), 28525),
 ((',:interp', 'o:prep', 'który:adj'), 28445),
 (('który:adj', 'mowa:subst', 'w:prep'), 28442),
 (('w:prep', 'usta:subst', '.:interp'), 23520),
 (('usta:subst', '.:interp', '1:adj'), 23346),
 (('.:interp', 'art:brev', '.:interp'), 22917),
 (('r:brev', '.:interp', 'nr:brev'), 17860)]

In [114]:
tag_trigrams_filtered_counter = Counter(
    (first, second, third)
    for tokens in tag_tokenized_acts.values()
    for (first, second, third) in zip(tokens, tokens[1:], tokens[2:])
    if is_word(first) and is_word(second) and is_word(third)
)

In [115]:
tag_trigrams_filtered_counter.most_common(10)

[(('o:prep', 'który:adj', 'mowa:subst'), 28525),
 (('który:adj', 'mowa:subst', 'w:prep'), 28442),
 (('mowa:subst', 'w:prep', 'usta:subst'), 13474),
 (('mowa:subst', 'w:prep', 'art:brev'), 12293),
 (('ustawa:subst', 'z:prep', 'dzień:subst'), 8589),
 (('właściwy:adj', 'do:prep', 'sprawa:subst'), 7964),
 (('minister:subst', 'właściwy:adj', 'do:prep'), 7886),
 (('w:prep', 'droga:subst', 'rozporządzenie:subst'), 4748),
 (('zastępować:fin', 'się:qub', 'wyraz:subst'), 3653),
 (('w:prep', 'ustawa:subst', 'z:prep'), 3646)]

## 12. Use PMI (with 5 occurrence threshold) to compute top 10 results for the trigrams. Devise a method for computing the values, based on the results for bigrams.

I am constructing _PMI_ of three outcomes $x$, $y$, $z$ to be:
$$
\text{pmi}(x; y; z) = \log_2 \frac{p(x, y, z)}{p(x)p(y)p(z)}
$$
For this constructions still holds that _PMI_ of outcomes measures the divergence between probability of their coincidence and the probability of their individual, independent occurences.

Saving counters to be used to calculate _PMI_ in a `pmi_trigram_mp.py` script that uses parallelization.

In [117]:
dump_counter(trigrams_counter, "trigrams_counter")
dump_counter(trigrams_filtered_counter, "trigrams_filtered_counter")

dump_counter(tag_trigrams_counter, "tag_trigrams_counter")
dump_counter(tag_trigrams_filtered_counter, "tag_trigrams_filtered_counter")

In [16]:
trigrams_counter = load_counter("trigrams_counter")
trigrams_filtered_counter = load_counter("trigrams_filtered_counter")

tag_trigrams_counter = load_counter("tag_trigrams_counter")
tag_trigrams_filtered_counter = load_counter("tag_trigrams_filtered_counter")

Loading calculated results.

In [27]:
trigrams_pmi = pickle.load(open("saved/trigram_pmi_results.p", "rb"))
tag_trigrams_pmi = pickle.load(open("saved/tag_trigram_pmi_results.p", "rb"))

### Top 10 results for trigrams
#### A. Non-lemmatized corpora
With any number of occurrences:

In [25]:
sorted(trigrams_pmi, key=lambda x: -x[1])[:10]

[(('english', 'language', 'college'), 44.952210959141496),
 (('wniebowzięcia', 'najświętszej', 'maryi'), 44.952210959141496),
 (('najświętszej', 'maryi', 'panny'), 44.952210959141496),
 (('porejestrowe', 'doświadczalnictwo', 'odmianowe'), 44.952210959141496),
 (('world', 'jewish', 'restitution'), 44.952210959141496),
 (('jewish', 'restitution', 'organisation'), 44.952210959141496),
 (('wszywanie', 'zamków', 'błyskawicznych'), 44.952210959141496),
 (('mit', 'beschrankter', 'haftung'), 44.952210959141496),
 (('prosimy', 'uważnie', 'przeczytać'), 44.952210959141496),
 (('aegroti', 'suprema', 'lex'), 44.952210959141496)]

With at lest 5 occurrences:

In [26]:
trigrams_pmi_5 = [(trigram, pmi) for trigram, pmi in trigrams_pmi if trigrams_counter[trigram] > 5]
sorted(trigrams_pmi_5, key=lambda x: -x[1])[:10]

[(('profilem', 'zaufanym', 'epuap'), 37.43585430543938),
 (('finałowego', 'turnieju', 'mistrzostw'), 37.0553740281969),
 (('potwierdzonym', 'profilem', 'zaufanym'), 36.762835628053026),
 (('piłce', 'nożnej', 'uefa'), 36.70428344569791),
 (('szybkiemu', 'postępowi', 'technicznemu'), 36.29043286136951),
 (('turnieju', 'mistrzostw', 'europy'), 36.28983928183392),
 (('kurtki', 'anorak', 'etc'), 35.907816839783045),
 (('poddanych', 'szybkiemu', 'postępowi'), 35.59528744289793),
 (('byłych', 'hitlerowskich', 'obozów'), 35.57391610322961),
 (('mln', 'dolarów', 'usa'), 35.47041152747575)]

#### B. Lemmatized and tagged corpora
With any number of occurrences:

In [28]:
sorted(tag_trigrams_pmi, key=lambda x: -x[1])[:10]

[(('english:subst', 'language:subst', 'college:subst'), 44.661591127103506),
 (('world:subst', 'jewish:subst', 'restitution:subst'), 44.661591127103506),
 (('jewish:subst', 'restitution:subst', 'organisation:subst'),
  44.661591127103506),
 (('wszywać:ger', 'zamek:subst', 'błyskawiczny:adj'), 44.661591127103506),
 (('şnapełniania:xxx', 'butel:subst', 'turystycznychş:xxx'),
  44.661591127103506),
 (('mitteldeutsche:subst', 'grundstucksgesellschaft:subst', 'mit:subst'),
  44.661591127103506),
 (('grundstucksgesellschaft:subst', 'mit:subst', 'beschrankter:subst'),
  44.661591127103506),
 (('mit:subst', 'beschrankter:subst', 'haftung:subst'), 44.661591127103506),
 (('ier:subst', 'ają:fin', 'cyc:subst'), 44.661591127103506),
 (('wl:subst', 'bezowdne:subst', 'cznik:subst'), 44.661591127103506)]

With at lest 5 occurrences:

In [30]:
tag_trigrams_pmi_5 = [(tag_trigram, pmi) for tag_trigram, pmi in tag_trigrams_pmi if tag_trigrams_counter[tag_trigram] > 5]
sorted(tag_trigrams_pmi_5, key=lambda x: -x[1])[:10]

[(('wymiennik:subst', 'przeponowy:adj', 'rurowy:adj'), 37.661591127103506),
 (('finałowy:adj', 'turniej:subst', 'mistrzostwa:subst'), 36.87368856771207),
 (('piłka:subst', 'nożny:adj', 'uefa:subst'), 35.90270169438254),
 (('turniej:subst', 'mistrzostwa:subst', 'europa:subst'), 35.672054706542426),
 (('przedwczesny:adj', 'wyrąb:subst', 'drzewostan:subst'), 35.44727200630274),
 (('kurtka:subst', 'anorak:subst', 'et\xa0cetera:brev'), 35.30403912248542),
 (('profil:subst', 'zaufany:adj', 'epuap:subst'), 35.07850835960057),
 (('milion:brev', 'dolar:subst', 'usa:subst'), 34.91675728960396),
 (('ja:ppron12', 'dopomóc:impt', 'bóg:subst'), 34.032234507023894),
 (('uniwersytet:subst', 'maria:subst', 'curie:subst'), 34.026780076931786)]

## 13. Create a table comparing the results for corpora with and without tagging and lemmatization (separate tables for bigrams and trigrams).

In [31]:
import pandas as pd

In [32]:
pmi_sort_fn = lambda x: -x[1]

A. Table for bigrams:


In [40]:
plain_bigrams_df = pd.DataFrame.from_records(
    data=[
        (first, second, pmi)
        for ((first, second), pmi) in sorted(bigrams_pmi_5, key=pmi_sort_fn)[:20]
    ],
    columns=["plain_first", "plain_second", "plain_pmi"],
)

tag_bigrams_df = pd.DataFrame.from_records(
    data=[
        (first, second, pmi)
        for ((first, second), pmi) in sorted(tag_bigrams_pmi_5, key=pmi_sort_fn)[:20]
    ],
    columns=["tagged_first", "tagged_second", "tagged_pmi"],
)

In [43]:
bigrams_df = pd.concat((plain_bigrams_df, tag_bigrams_df), axis=1)
bigrams_df.columns = bigrams_df.columns.str.split("_", expand=True)

bigrams_df

plain                                           tagged  \
                 first           second        pmi                first   
0            diagności    laboratoryjni  19.891143           adam:subst   
1             odczynów  poszczepiennych  19.891143       piotrków:subst   
2                adama      mickiewicza  19.891143       przeponowy:adj   
3             papierem     wartościowym  19.668751  chrześcijanin:subst   
4             piotrków      trybunalski  19.668751          maria:subst   
5           lambrekiny          okienne  19.668751           hugo:subst   
6               schedę         spadkową  19.668751        tadeusz:subst   
7               buraka        cukrowego  19.476105      upośledzony:adj   
8             zdrowego            stylu  19.476105         jedwab:subst   
9      zapieczętowanej         kopercie  19.446358          vitis:subst   
10              moszcz          gronowy  19.306180   nietykalność:subst   
11  zniekształcających           rzeźbę  19.306180      lambrekin:subst   
12                ulgę       uczniowską  19.253713      chłodnica:subst   
13          wyznaniowa         żydowska  19.154177         michał:subst   
14           małżeńską      wspólnością  19.136255         zatoka:subst   
15           prewencję          rentową  19.113535          dolar:subst   
16            ziemiach       zachodnich  19.016674          zaufany:adj   
17             sieroty         zupełnej  19.016674            ter:subst   
18           motywacji     zasługującej  19.016674         odczyn:subst   
19             partiom      politycznym  19.016674       radosław:subst   

                                  
               second        pmi  
0    mickiewicz:subst  19.745833  
1     trybunalski:adj  19.523441  
2          rurowy:adj  19.330796  
3      baptysta:subst  19.108403  
4         curie:subst  19.008867  
5      kołłątaj:subst  19.008867  
6   kotarbiński:subst  19.008867  
7        umysłowo:adv  18.871364  
8       wiskozowy:adj  18.871364  
9      vinifera:subst  18.871364  
10       cielesny:adj  18.816222  
11        okienny:adj  18.786475  
12     odmulina:subst  18.745833  
13   oczapowski:subst  18.745833  
14        gdański:adj  18.646297  
15          usa:subst  18.630356  
16        epuap:subst  18.630356  
17     torialnego:adj  18.523441  
18  poszczepienny:adj  18.523441  
19     sikorski:subst  18.523441

B. Table for trigrams:

In [44]:
plain_trigrams_df = pd.DataFrame.from_records(
    data=[
        (first, second, third, pmi)
        for ((first, second, third), pmi) in sorted(trigrams_pmi_5, key=pmi_sort_fn)[:20]
    ],
    columns=["plain_first", "plain_second", "plain_third", "plain_pmi"],
)

tag_trigrams_df = pd.DataFrame.from_records(
    data=[
        (first, second, third, pmi)
        for ((first, second, third), pmi) in sorted(tag_trigrams_pmi_5, key=pmi_sort_fn)[:20]
    ],
    columns=["tag_first", "tag_second", "tag_third", "tag_pmi"],
)

In [45]:
trigrams_df = pd.concat((plain_trigrams_df, tag_trigrams_df), axis=1)
trigrams_df.columns = trigrams_df.columns.str.split("_", expand=True)

trigrams_df

plain                                               \
               first              second         third        pmi   
0           profilem            zaufanym         epuap  37.435854   
1         finałowego            turnieju    mistrzostw  37.055374   
2      potwierdzonym            profilem      zaufanym  36.762836   
3              piłce              nożnej          uefa  36.704283   
4          szybkiemu           postępowi  technicznemu  36.290433   
5           turnieju          mistrzostw        europy  36.289839   
6             kurtki              anorak           etc  35.907817   
7          poddanych           szybkiemu     postępowi  35.595287   
8             byłych       hitlerowskich        obozów  35.573916   
9                mln             dolarów           usa  35.470412   
10         religijne        uroczystości    pogrzebowe  35.363496   
11            trwale  zniekształcających        rzeźbę  35.274491   
12         komunalne               osady      ściekowe  35.223319   
13         wybitnych             młodych     naukowców  35.175367   
14      zubożających             warstwę       ozonową  35.164308   
15          wydatkom            bieżącym    ponoszonym  35.045320   
16    międzynarodowy              sygnał    wywoławczy  35.036332   
17  socjalistycznych            republik   radzieckich  35.027398   
18        kolektywną             pozycję      znaczącą  34.913463   
19     hitlerowskich              obozów       zagłady  34.883601   

                  tag                                                     
                first               second              third        pmi  
0     wymiennik:subst       przeponowy:adj         rurowy:adj  37.661591  
1        finałowy:adj        turniej:subst  mistrzostwa:subst  36.873689  
2         piłka:subst            nożny:adj         uefa:subst  35.902702  
3       turniej:subst    mistrzostwa:subst       europa:subst  35.672055  
4    przedwczesny:adj          wyrąb:subst   drzewostan:subst  35.447272  
5        kurtka:subst         anorak:subst     et cetera:brev  35.304039  
6        profil:subst          zaufany:adj        epuap:subst  35.078508  
7         milion:brev          dolar:subst          usa:subst  34.916757  
8          ja:ppron12         dopomóc:impt          bóg:subst  34.032235  
9   uniwersytet:subst          maria:subst        curie:subst  34.026780  
10         trwale:adv   zniekształcać:pact       rzeźba:subst  33.761479  
11       zubożać:pact        warstwa:subst        ozonowy:adj  33.667238  
12         wielki:adj       księstwo:subst   luksemburg:subst  33.514750  
13    niepożądany:adj         odczyn:subst  poszczepienny:adj  33.188900  
14    hitlerowski:adj           obóz:subst      zagłada:subst  33.032235  
15    królestwo:subst           wielki:adj     brytania:subst  32.885393  
16        filia:subst   politechnika:subst         łódzki:adj  32.884336  
17  diagnostyka:subst             in:subst        vitro:subst  32.485418  
18      kościół:subst  chrześcijanin:subst     baptysta:subst  32.343802  
19     włóknina:subst            nawet:qub   impregnować:ppas  32.209736

## 14. Answer the following questions:

### 1. Why do we have to filter the bigrams, rather than the token sequence?
Because otherwise some bigrams would not contain subsequent tokens.

### 1. Which method works better for the bigrams and which for the trigrams?
In my opinion the method using tagging and lemmatization works better for bigrams and the method not using tagging and lemmatization for trigrams.

When analysing bigrams, if not using tagging and lemmatization, the results show pair of words that were used in particular inflectional form, such as `wyznaniowa żydowska`. By using tagging and lemmatization we find pair of words that are correlated in more general manner, in any form, such as `Adam Mickiewicz` or `zaufany epuap`.

With trigrams though, when using tagging, the pmi score is downgraded visibly because the tokens that form the trigram can be used in any possible form thus its probability is lower. Without using tagging we find trigrams that are more specific in case of form but with less distorted per-token probability. For example `profil zaufany epuap` is much lower in tagged method, while `wymiennik przeponowy nowy` is only visible in tagged method.

### 3. What types of expressions are discovered by the methods.

Both methods find mainly n-grams built only from nouns.

### 4. Can you devise a different type of filtering that would yield better results?

I can propose a metric _gap pointwise mutual information_ which would calculate the divergance between probability of two words appearing in an n-gram at any positions and the probability of their occurrence but assuming independence. The metric would be higher for pair of words that often appear together, but possibly in a bigger scope, e.g. words `algorytm` and `maszynowego` in phrase `algorytm uczenia maszynowego`.